In [13]:
import geopandas as gpd
import rasterio
import numpy as np
import os

# Paths
raster_folder_path = r"C:\Users\FIqbal\Downloads\Australia_GISdata_LTAym_YearlyMonthlyTotals_GlobalSolarAtlas-v2_GEOTIFF\Australia_GISdata_LTAy_YearlyMonthlyTotals_GlobalSolarAtlas-v2_GEOTIFF"
output_folder = r"C:\Users\FIqbal\Downloads\Output"
output_suitability_path = os.path.join(output_folder, "suitability_map.tif")

# List of rasters and their corresponding weights
raster_files = [
    ("TEMP.tif", 0.2), 
    ("PVOUT.tif", 0.2), 
    ("OPTA.tif", 0.1), 
    ("GTI.tif", 0.1), 
    ("GHI.tif", 0.1), 
    ("DNI.tif", 0.1), 
    ("DIF.tif", 0.1), 
    ("Aspect.tif", 0.05), 
    ("Slope.tif", 0.05)
]

# Function to read and normalize raster data
def read_and_normalize_raster(raster_path):
    with rasterio.open(raster_path) as src:
        data = src.read(1)  # Read the first band
        data = np.where(data == src.nodata, np.nan, data)  # Mask nodata
        return (data - np.nanmin(data)) / (np.nanmax(data) - np.nanmin(data))  # Normalize

# Initialize suitability score
first_raster_path = os.path.join(raster_folder_path, raster_files[0][0])
suitability_score = np.zeros_like(read_and_normalize_raster(first_raster_path))

# Loop through raster files and calculate the weighted sum
for raster_file, weight in raster_files:
    raster_path = os.path.join(raster_folder_path, raster_file)
    normalized_data = read_and_normalize_raster(raster_path)
    suitability_score += normalized_data * weight  # Weighted sum

# Save the suitability map
with rasterio.open(
    output_suitability_path,
    'w',
    driver='GTiff',
    height=suitability_score.shape[0],
    width=suitability_score.shape[1],
    count=1,
    dtype='float32',
    crs='EPSG:4326',  # Replace with your CRS if different
    transform=rasterio.transform.from_origin(0, 0, 1, 1)  # Replace with actual transform
) as dst:
    dst.write(suitability_score, 1)

print(f"Suitability map saved to {output_suitability_path}")


ValueError: operands could not be broadcast together with shapes (4200,5760) (1050,1440) (4200,5760) 

In [16]:
import geopandas as gpd
import rasterio
import rasterio.mask
import numpy as np
import os

# Paths
raster_folder_path = r"C:\Users\FIqbal\Downloads\Australia_GISdata_LTAym_YearlyMonthlyTotals_GlobalSolarAtlas-v2_GEOTIFF\Australia_GISdata_LTAy_YearlyMonthlyTotals_GlobalSolarAtlas-v2_GEOTIFF"
output_folder = r"C:\Users\FIqbal\Downloads\Output"
shapefile_path = r"C:\Users\FIqbal\Downloads\Data\Property_SHapefile.shp"

# List of rasters to process with corresponding weights
raster_files = [
    ("TEMP.tif", 0.2), 
    ("PVOUT.tif", 0.3), 
    ("OPTA.tif", 0.1), 
    ("GTI.tif", 0.2), 
    ("GHI.tif", 0.1), 
    ("DNI.tif", 0.05), 
    ("DIF.tif", 0.05),
    ("Aspect.tif", 0.025), 
    ("Slope.tif", 0.025)
]

# Function to read and normalize raster data
def read_and_normalize_raster(raster_path):
    with rasterio.open(raster_path) as src:
        data = src.read(1)  # Read the first band
        data = np.where(data == src.nodata, np.nan, data)  # Mask nodata
        return data

# Function to resample raster data to match a target shape
def resample_to_shape(data, target_shape):
    if data.shape != target_shape:
        return np.resize(data, target_shape)  # Simple resizing; for more accurate resampling, consider using rasterio.warp
    return data

# Initialize suitability score
suitability_score = None
target_shape = None

# Loop through each raster file
for raster_file, weight in raster_files:
    raster_path = os.path.join(raster_folder_path, raster_file)
    
    # Read and normalize raster
    normalized_data = read_and_normalize_raster(raster_path)
    
    # Set target shape based on the first raster
    if target_shape is None:
        target_shape = normalized_data.shape
    
    # Resample if necessary
    normalized_data = resample_to_shape(normalized_data, target_shape)
    
    # Initialize suitability score array if not already done
    if suitability_score is None:
        suitability_score = np.zeros(target_shape)
    
    # Weighted sum for suitability score
    suitability_score += normalized_data * weight

# Save the suitability map
suitability_output_path = os.path.join(output_folder, "suitability_map.tif")
with rasterio.open(
    suitability_output_path, 'w',
    driver='GTiff',
    height=suitability_score.shape[0],
    width=suitability_score.shape[1],
    count=1,
    dtype=suitability_score.dtype,
    crs='EPSG:4326',  # Adjust CRS if necessary
    transform=rasterio.transform.from_origin(0, 0, 1, 1)  # Adjust as per your data
) as dst:
    dst.write(suitability_score, 1)

print(f'Suitability map saved to: {suitability_output_path}')


C:\Users\FIqbal\AppData\Roaming\Python\Python311\site-packages\rasterio\__init__.py:327: NotGeoreferencedWarning: The given matrix is equal to Affine.identity or its flipped counterpart. GDAL may ignore this matrix and save no geotransform without raising an error. This behavior is somewhat driver-specific.
  dataset = writer(


Suitability map saved to: C:\Users\FIqbal\Downloads\Output\suitability_map.tif
